# Stacks and Queues

In [1]:
from typing import Deque, List, Optional
from collections import deque, namedtuple
from dataclasses import dataclass

from utils import run_tests

## Stacks
- have LIFO (last-in, first-out) structure

### Tips
- Learn to recognize when the stack **LIFO** property is **applicable**. For example, **parsing** typically benefits from a stack.
- Consider **augmenting** the basic stack or queue data structure to support additional operations, such as finding the maximum element.

### Libraries
Use a list if not required to implement your own stack   
Note: When called on an empty list, both stack[-1] and stack.pop() raise an IndexError exception

In [2]:
stack = [1, 2, 3]
print('stack:                                          ', stack)

print('push an element on to the stack: stack.append():', stack.append(5))

print('retrieve without removing from stack: stack[-1]:', stack[-1])

print('remove top element from stack: stack.pop():     ', stack.pop())

print('test if stack is empty: len(stack) == 0         ', len(stack) == 0)


stack:                                           [1, 2, 3]
push an element on to the stack: stack.append(): None
retrieve without removing from stack: stack[-1]: 5
remove top element from stack: stack.pop():      5
test if stack is empty: len(stack) == 0          False


## Queues
- have FIFO (First-in, first-out) structure

### Tips
- Learn to recognize when the stack **LIFO** property is **applicable**. For example, queues are ideal when **order** needs to be **preserved**

### Libraries
Use a collections.deque class

In [5]:
queue = deque(range(5))
print('Queue:                                                  ', queue)

print('push an element on to queue: queue.append():            ', queue.append(10))   # adds to end

print('retrieve but not remove: queue[0]:                      ', queue[0])

print('retrieve and remove element from queue: queue.popleft():', queue.popleft())   # remove from front

print('Queue:                                                  ', queue)

print('to remove from end: pop():                              ', queue.pop())       # remove from end

print('add to front: appendleft():                             ', queue.appendleft(100))

print('Queue:                                                  ', queue)


Queue:                                                   deque([0, 1, 2, 3, 4])
push an element on to queue: queue.append():             None
retrieve but not remove: queue[0]:                       0
retrieve and remove element from queue: queue.popleft(): 0
Queue:                                                   deque([1, 2, 3, 4, 10])
to remove from end: pop():                               10
add to front: appendleft():                              None
Queue:                                                   deque([100, 1, 2, 3, 4])


### Queue Class

In [7]:
class Queue:

    def __init__(self) -> None:
        self._queue: Deque[int] = deque()
    
    def is_empty(self) -> bool:
        return len(self._queue) == 0

    def enqueue(self, data: int) -> None:
        self._queue.append(data)

    def dequeue(self) -> int:
        return self._queue.popleft()

    def max(self) -> int:
        return max(self._queue)

In [8]:
q = Queue()
print(q.is_empty())
for i in [1, 4, 5, 6, 1, 2, 10]:
    q.enqueue(i)
    print('max:', q.max())
print(q.dequeue())
print(q.max())
print(q.dequeue())

True
max: 1
max: 4
max: 5
max: 6
max: 6
max: 6
max: 10
1
10
4


## Stack Problems

### 8.1: Implement a Stack with a Max API

In [15]:
class StackMax:

    ElementWithCachedMax = namedtuple('ElementWithCachedMax', ['element', 'max'])

    def __init__(self) -> None:
        self._stack: List[StackMax.ElementWithCachedMax] = []

    def is_empty(self) -> bool:
        return len(self._stack) == 0

    def pop(self) -> int:
        return self._stack.pop().element

    def max(self) -> int:
        return self._stack[-1].max 

    def push(self, num: int) -> None:
        self._stack.append(
            self.ElementWithCachedMax(
                num, num if self.is_empty() else max([num, self.max()])
                )
        )


Each method is $O(1)$ time complexity. $O(n)$ space complexity.

In [19]:
s = StackMax()
print(s.is_empty())
print(s._stack)
for i in reversed(range(10)):
    s.push(i)
print(s._stack)
print(s.max())
print(s.pop())
print(s.max())

print()

s = StackMax()
print(s.is_empty())
print(s._stack)
for i in [2, 2, 1, 1, 5, 3, 2, 0]:
    s.push(i)
print(s._stack)
print(s.max())
print(s.pop())
print(s.max())

True
[]
[ElementWithCachedMax(element=9, max=9), ElementWithCachedMax(element=8, max=9), ElementWithCachedMax(element=7, max=9), ElementWithCachedMax(element=6, max=9), ElementWithCachedMax(element=5, max=9), ElementWithCachedMax(element=4, max=9), ElementWithCachedMax(element=3, max=9), ElementWithCachedMax(element=2, max=9), ElementWithCachedMax(element=1, max=9), ElementWithCachedMax(element=0, max=9)]
9
0
9

True
[]
[ElementWithCachedMax(element=2, max=2), ElementWithCachedMax(element=2, max=2), ElementWithCachedMax(element=1, max=2), ElementWithCachedMax(element=1, max=2), ElementWithCachedMax(element=5, max=5), ElementWithCachedMax(element=3, max=5), ElementWithCachedMax(element=2, max=5), ElementWithCachedMax(element=0, max=5)]
5
0
5


#### Variant: Improve space complexity for many duplicate entries
each element on stack has a max value in stack but this wastes space when there are a ton of duplicates

In [34]:
class StackMaxDuplicates:


    @dataclass
    class CachedMaxCount:
        max: int
        count: int = 1

        def increment(self) -> None:
            self.count += 1
        
        def decrement(self) -> None:
            self.count -= 1


    def __init__(self) -> None:
        self._stack: List[int] = []
        self._max_stack: List[StackMaxDuplicates.CachedMaxCount] = []

    def is_empty(self) -> bool:
        return len(self._stack) == 0

    def pop(self) -> int:
        # update max stack count
        self._max_stack[-1].decrement()
        if self._max_stack[-1].count == 0:
            self._max_stack.pop()
        return self._stack.pop()

    def max(self) -> int:
        return self._max_stack[-1].max 

    def push(self, num: int) -> None:
        # update max stack
        if self.is_empty() or num > self._max_stack[-1].max:
            self._max_stack.append(self.CachedMaxCount(num))
        else:
            self._max_stack[-1].increment()
        
        # add elemnt to stack
        self._stack.append(num)


In [38]:
s = StackMaxDuplicates()
print(s.is_empty())
for i in [2, 2, 1, 1, 5, 3, 2, 0, 10]:
    s.push(i)
print(s._stack)
print(s._max_stack)
while not s.is_empty():
    m = s.max()
    num = s.pop()
    print('element:', num, 'max:', m)

True
[2, 2, 1, 1, 5, 3, 2, 0, 10]
[StackMaxDuplicates.CachedMaxCount(max=2, count=4), StackMaxDuplicates.CachedMaxCount(max=5, count=4), StackMaxDuplicates.CachedMaxCount(max=10, count=1)]
element: 10 max: 10
element: 0 max: 5
element: 2 max: 5
element: 3 max: 5
element: 5 max: 5
element: 1 max: 2
element: 1 max: 2
element: 2 max: 2
element: 2 max: 2


### 8.2: Evaluate RPN Expressions
Reverse Polish Notation:
- Is is a single digit or a sequence of digits, prefixed with an option '-', e.g., '-5', '9', '123'
- It is of the form "A,B,@" where @ is one of '+', '-', 'x', '/'
e.g.: '1729', '3,4,+,2,x,1,+', '1,1,+,-2,x'   
Can be evaluate uniquely to an integer   
Divsion is floor division

In [39]:
def evaluate_rpn(expression: str, delimiter: str=',') -> int:

    operations = {
                    '+': lambda x, y: x + y,
                    '-': lambda x, y: x - y,
                    'x': lambda x, y: x * y,
                    '/': lambda x, y: x // y,
    }

    rpn = expression.split(delimiter)
    stack = []
    for s in rpn:
        if s in operations:
            x, y = stack.pop(), stack.pop()
            stack.append(operations[s](x, y))
        else:
            stack.append(int(s))

    return stack.pop()
        
inputs, outputs = ('1729', '5,3,+', '3,4,+,2,x,1,+', '1,1,+,-2,x'), (1729, 8, 15, -4)
run_tests(evaluate_rpn, inputs, outputs)

$O(n)$ time complexity

#### Variant: Rule 2 operation is flipped
i.e. *,A,B

In [41]:
def evaluate_rpn_flipped(expression: str, delimiter: str=',') -> int:

    operations = {
                    '+': lambda x, y: x + y,
                    '-': lambda x, y: x - y,
                    'x': lambda x, y: x * y,
                    '/': lambda x, y: x // y,
    }

    rpn = expression.split(delimiter)

    if len(rpn) == 1:
        return int(rpn[0])

    operation, A, B = rpn[0:3]
    result = operations[operation](int(A), int(B))

    if len(rpn) > 3:
        i = 3
        while i < len(rpn):
            operation, num = rpn[i], int(rpn[i+1])
            result = operations[operation](result, num)
            i += 2

    return result
        
inputs, outputs = ('1729', '+,5,3', '+,3,4,x,2,+,1', '+,1,1,x,-2'), (1729, 8, 15, -4)
run_tests(evaluate_rpn_flipped, inputs, outputs)

### 8.3: Is a String Well-Formed?
A string over the characters "[](){}" is well-formed if the different types of brackets match in the correct order

In [52]:
def is_well_formed(s: str) -> int:

    lookup = {
                        '[': ']',
                        '(': ')',
                        '{': '}'
    }
    left_brackets = []
    for token in s:
        if token in lookup:
            left_brackets.append(token)
        else:
            if len(left_brackets) == 0 or token != lookup[left_brackets.pop()]:
                return False 
    return True 

inputs, outputs = ('[]{}()', '[{()}]', '][()', '[[)]', '([]){()}', '[()[]{()()}]'), (True, True, False, False, True, True)
run_tests(is_well_formed, inputs, outputs)

$O(n)$ time complexity

In [53]:
is_well_formed('[]()')

True

### 8.4: Normalize Pathnames

### 8.5: Compute Buildings with a Sunset View

## Queue Problems

### 8.6: Compute Binary Tree Nodes in Order of Increasing Depth

### 8.7: Implement a Circular Queue

In [28]:
class CircularQueue:
    SCALE_FACTOR = 2

    def __init__(self, capacity: int) -> None:
        self.capacity: int = capacity
        self.size: int = 0
        self.head: int = 0
        self.tail: int = 0
        self.queue: List[int] = [-1] * capacity

    def is_empty(self) -> bool:
        return self.size == 0

    def enqueue(self, data: int) -> None:
        if self.size == self.capacity:
            self._grow_array()

        self.queue[self.tail] = data 
        self.tail = (self.tail + 1) % self.capacity
        self.size += 1

    def dequeue(self) -> int:
        if self.size == 0:
            raise IndexError('Queue is empty')
        if self.size < self.capacity / (self.SCALE_FACTOR * self.SCALE_FACTOR):
            self._shrink_array()
        val = self.queue[self.head]
        self.head = (self.head + 1) % self.capacity
        self.size -= 1
        return val


    def _grow_array(self) -> None:
        # put elements consequtively
        # items at end could come before head
        self.queue = self.queue[self.head:] + self.queue[:self.head]

        # reset head and tail
        self.head = 0
        self.tail = self.size

        self.capacity *= self.SCALE_FACTOR
        self.queue += [-1] * (self.capacity // self.SCALE_FACTOR)
        
    
    # def _shrink_array(self) -> None:
    #     # put elements consequtively
    #     # items at end could come before head
    #     self.queue = self.queue[self.head:] + self.queue[:self.head]

    #     # reset head and tail
    #     self.head = 0
    #     self.tail = self.size

    #     self.capacity //= (self.SCALE_FACTOR * self.SCALE_FACTOR)
    #     self.queue = self.queue[:self.capacity]
        

        


Time complexity of dequeue is $O(1)$ and the amortized time of enqueue is $O(1)$

In [34]:
q = CircularQueue(4)
for i in range(6):
    q.enqueue(i)
print(q.queue)
for i in range(6):
    print(q.dequeue())
print(q.queue)
print(q.capacity)

[0, 1, 2, 3, 4, 5, -1, -1]
0
1
2
3
4
5
[5, -1]
2


### 8.8: Implement a Queue Using a Stack

In [43]:
class QueueOfStacks:

    def __init__(self) -> None:
        self._enqueue_stack: List[int] = []
        self._dequeue_stack: List[int] = []

    def is_empty(self) -> bool:
        return len(self._enqueue_stack) == 0 and len(self._dequeue_stack) == 0

    def enqueue(self, data: int) -> None:
        self._enqueue_stack.append(data)

    def dequeue(self) -> int:

        # transfer elements for enqueue to dequeue stack
        if len(self._dequeue_stack) == 0:
            while self._enqueue_stack:
                self._dequeue_stack.append(self._enqueue_stack.pop())
                
        return self._dequeue_stack.pop()
        

In [44]:
q = QueueOfStacks()
print(q.is_empty())
for i in range(5):
    q.enqueue(i)
print(q.dequeue())
q.enqueue(100)
while not q.is_empty():
    print(q.dequeue())

True
0
1
2
3
4
100


Takes $O(m)$ time where $m$ is number of operations since each element is pushed/popped no more than twice

### 8.9: Queue with Max API